In [1]:
import os
import skimage.io as io
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import concatenate, Conv2DTranspose, Input, Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization,GlobalMaxPooling2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam, SGD

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13913514707045813925
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9920577536
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15251182279767660702
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:07:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

!mkdir -p data

!cd data && wget http://images.cocodataset.org/zips/train2017.zip

!cd data && unzip -q train2017.zip

!cd data && rm train2017.zip

!cd data && wget http://images.cocodataset.org/zips/val2017.zip 

!cd data && unzip -q val2017.zip

!cd data && rm val2017.zip

!cd data && wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

!cd data && unzip -q annotations_trainval2017.zip

!cd data && rm annotations_trainval2017.zip

!conda install -c conda-forge pycocotools

!ls data/

In [3]:
COCO_ROOT = './data/'
import sys
#sys.path.insert(0, os.path.join(COCO_ROOT, 'cocoapi/PythonAPI'))
from pycocotools.coco import COCO

In [4]:
class Dataset():

    def crop_images(self, img, inp_size, random_crop=False):
        shape = tf.shape(img)
        pad = (
            [0, tf.maximum(inp_size - shape[0], 0)],
            [0, tf.maximum(inp_size - shape[1], 0)],
            [0, 0],
        )
        img = tf.pad(img, pad)

        if random_crop:
            img = tf.image.random_crop(img, (inp_size, inp_size, shape[2]))
        else: # central crop
            shape = tf.shape(img)
            ho = (shape[0] - inp_size) // 2
            wo = (shape[1] - inp_size) // 2
            img = img[ho:ho+inp_size, wo:wo+inp_size, :]

        return img

    def train_dataset(self, batch_size, epochs, inp_size):

        def item_to_images(item):
            random_crop = True
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            img_combined = self.crop_images(img_combined, inp_size, random_crop)

            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class

        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        dataset = dataset.shuffle(buffer_size=len(self.img_list))
        dataset = dataset.map(item_to_images)
        #dataset = dataset.repeat(epochs)
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset

    def val_dataset(self, batch_size, inp_size):

        def item_to_images(item):
            random_crop = False
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            img_combined = self.crop_images(img_combined, inp_size, random_crop)

            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class

        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        dataset = dataset.map(item_to_images)
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset

In [5]:
class COCO_Dataset(Dataset):

    def __init__(self, sublist):
        ann_file_fpath = os.path.join(COCO_ROOT, 'annotations', 'instances_'+sublist+'2017.json')
        self.coco = COCO(ann_file_fpath)
        self.cat_ids = self.coco.getCatIds(catNms=['person'])
        self.img_list = self.coco.getImgIds(catIds=self.cat_ids)

    def read_images(self, img_id):
        img_id = int(img_id.numpy())
        img_data = self.coco.loadImgs(img_id)[0]
        img_fname = '/'.join(img_data['coco_url'].split('/')[-2:])

        img = io.imread(os.path.join(COCO_ROOT, img_fname))
        if len(img.shape) == 2:
            img = np.tile(img[..., None], (1, 1, 3))

        ann_ids = self.coco.getAnnIds(imgIds=img_data['id'], catIds=self.cat_ids, iscrowd=None)
        anns = self.coco.loadAnns(ann_ids)
        mask_class = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        for i in range(len(anns)):
            mask_class += self.coco.annToMask(anns[i])
        mask_class = (mask_class > 0).astype(np.uint8)

        img_combined = np.concatenate([img, mask_class[..., None]], axis=2)

        return img_combined

In [6]:
COCO_dataset_train = COCO_Dataset('train')
COCO_dataset_val = COCO_Dataset('val')

loading annotations into memory...
Done (t=12.77s)
creating index...
index created!
loading annotations into memory...
Done (t=1.79s)
creating index...
index created!


In [7]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 3
BATCH_SIZE = 64
EPOCHS = 5

In [8]:
train_ds = COCO_dataset_train.train_dataset(BATCH_SIZE, EPOCHS, IMG_HEIGHT)
val_ds = COCO_dataset_val.val_dataset(BATCH_SIZE, IMG_HEIGHT)

In [9]:
!pip install -U efficientnet==0.0.4

!pip install git+https://github.com/tensorflow/examples.git
!pip install -U tfds-nightly

In [10]:
from tensorflow_examples.models.pix2pix import pix2pix

In [11]:
import tensorflow.keras.applications.efficientnet as efn

In [12]:
model = efn.EfficientNetB4(weights='imagenet', input_shape=[256, 256, 3], include_top=False)

In [13]:
model.trainable = False

In [14]:
up_stack = [
    pix2pix.upsample(224, 3),  # 8x8 -> 16x16
    pix2pix.upsample(80, 3),  # 16x16 -> 32x32
    pix2pix.upsample(48, 3),  # 32x32 -> 64x64
    pix2pix.upsample(32, 3),   # 64x64 -> 128x128
]

In [15]:
layer_names = [
    'stem_conv',     # 128x128 48
    'block2b_add',   # 64x64 32
    'block3b_add',   # 32x32 56
    'block5b_add',   # 16x16 160
    #'block6b_add ',  # 8x8 272
    'top_activation',      # 8x8 1792 (top)
]

In [16]:
layers = [model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=model.input, outputs=layers)

down_stack.trainable = False

In [17]:
def unet_model(output_channels):
    inputs = tf.keras.layers.Input(shape=[256, 256, 3]) 
    x = inputs

    # Downsampling through the model
    skips = down_stack(x)
    x = skips[-1]

    #x = tf.keras.layers.Conv2D(1024, 3, strides=(1, 1), padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D(512, 3, strides=(1, 1), padding='same', activation='swish', kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Conv2D(256, 3, strides=(1, 1), padding='same', activation='swish', kernel_initializer='he_normal')(x)

    x = up_stack[0](x)
    x = tf.keras.layers.Concatenate()([x, skips[3]])
    x = tf.keras.layers.Conv2D(128, 3, strides=(1, 1), padding='same', activation='swish', kernel_initializer='he_normal')(x)


    x = up_stack[1](x)
    x = tf.keras.layers.Concatenate()([x, skips[2]])
    x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='swish', kernel_initializer='he_normal')(x)


    x = up_stack[2](x)
    x = tf.keras.layers.Concatenate()([x, skips[1]])
    x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='swish', kernel_initializer='he_normal')(x)


    x = up_stack[3](x)
    x = tf.keras.layers.Concatenate()([x, skips[0]])
    x = tf.keras.layers.Conv2D(32, 3, strides=(1, 1), padding='same', activation='swish', kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Conv2D(16, 3, strides=(1, 1), padding='same', activation='swish', kernel_initializer='he_normal')(x)


    # This is the last layer of the model
    #last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same')  #64x64 -> 128x128

  
    last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same', activation='sigmoid')  #128x128 -> 256x256
    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [18]:
OUTPUT_CHANNELS = 1
model = unet_model(OUTPUT_CHANNELS)

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             [(None, 128, 128, 4  17673823    ['input_2[0][0]']                
                                8),                                                               
                                 (None, 64, 64, 32)                                               
                                , (None, 32, 32, 56                                               
                                ),                                                          

In [20]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = 'accuracy')

hist = model.fit(train_ds, epochs = EPOCHS, batch_size = BATCH_SIZE, validation_data = val_ds)

Epoch 1/5
1001/1001 [==============================] - 900s 885ms/step - loss: 0.4520 - accuracy: 0.7837 - val_loss: 0.5056 - val_accuracy: 0.7466
Epoch 2/5
1001/1001 [==============================] - 722s 721ms/step - loss: 0.4299 - accuracy: 0.7946 - val_loss: 0.6405 - val_accuracy: 0.7554
Epoch 3/5
1001/1001 [==============================] - 714s 714ms/step - loss: 0.4179 - accuracy: 0.8023 - val_loss: 0.6210 - val_accuracy: 0.7455
Epoch 4/5
1001/1001 [==============================] - 719s 719ms/step - loss: 0.4112 - accuracy: 0.8059 - val_loss: 0.5164 - val_accuracy: 0.7473
Epoch 5/5
1001/1001 [==============================] - 714s 714ms/step - loss: 0.4060 - accuracy: 0.8080 - val_loss: 0.4693 - val_accuracy: 0.7796


In [22]:
model.evaluate(val_ds)

42/42 [==============================] - 27s 641ms/step - loss: 0.4693 - accuracy: 0.7796


[0.46930262446403503, 0.7795562744140625]